This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/intro.html).  I recommend having both windows open, side-by-side!

## Instructions for running this notebook on Deepnote

- Log in (the free account will be sufficient for this class)
- "Duplicate" the document.  Icon is in the top right next to Login.
- Setup the "Environment" ("Environment" icon on the left of this window).  
    - Click "Set up a new Docker image" and point it to `russtedrake/manipulation:latest` .
- Run all of the cells (can use the "Run notebook" icon just above this cell)
- Open a different browser window to the "ngrok" url printed out in the 3rd cell to open meshcat.
- In meshcat, click "Open controls" and use the sliders that appear to teleop.
- When you're done, click the "Stop Simulation" button in meshcat.

I'll make all of this a little smoother in time for the class.  [#148](https://github.com/RussTedrake/manipulation/issues/148), [#149](https://github.com/RussTedrake/manipulation/issues/149)

In [ ]:
# Imports
import numpy as np

from pydrake.examples.manipulation_station import ManipulationStation
from pydrake.geometry import MeshcatVisualizerCpp
from pydrake.manipulation.planner import (  
  DifferentialInverseKinematicsParameters, 
  DifferentialInverseKinematicsIntegrator )
from pydrake.math import RigidTransform, RotationMatrix
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.jupyter_widgets import PoseSliders, WidgetSystem

from manipulation import running_as_notebook
from manipulation.meshcat_cpp_utils import StartMeshcat

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

# Teleop Example (3D)

In this example, we assemble a diagram with all of the relevant subsystems (the manipulation station, the meshcat visualizer, and some systems that provide a minimal teleop interface and convert the teleop output from end-effector commands into joint commands.  We'll learn more about each of these components in the following chapters.

**NOTE:** If you command the robot to move its gripper beyond what is possible, then you get a message about "differential IK" failing.  I've left that in for now (rather than setting very conservative slider limits) partly because it has tutorial value.  We'll understand it more precisely soon!  For now, just stop the simulation and rerun the cell if you get stuck.


In [ ]:
# Some GUI code that will be moved into Drake.

from collections import namedtuple
from pydrake.common.value import AbstractValue
from pydrake.math import RollPitchYaw
from pydrake.systems.framework import LeafSystem, PublishEvent

class MeshcatPoseSliders(LeafSystem):
    """
    Provides a set of ipywidget sliders (to be used in a Jupyter notebook) with
    one slider for each of roll, pitch, yaw, x, y, and z.  This can be used,
    for instance, as an interface to teleoperate the end-effector of a robot.

    .. pydrake_system::

        name: PoseSliders
        output_ports:
        - pose
    """
    # TODO(russt): Use namedtuple defaults parameter once we are Python >= 3.7.
    Visible = namedtuple("Visible", ("roll", "pitch", "yaw", "x", "y", "z"))
    Visible.__new__.__defaults__ = (True, True, True, True, True, True)
    MinRange = namedtuple("MinRange", ("roll", "pitch", "yaw", "x", "y", "z"))
    MinRange.__new__.__defaults__ = (-np.pi, -np.pi, -np.pi, -1.0, -1.0, -1.0)
    MaxRange = namedtuple("MaxRange", ("roll", "pitch", "yaw", "x", "y", "z"))
    MaxRange.__new__.__defaults__ = (np.pi, np.pi, np.pi, 1.0, 1.0, 1.0)
    Value = namedtuple("Value", ("roll", "pitch", "yaw", "x", "y", "z"))
    Value.__new__.__defaults__ = (0.0, 0.0, 0.0, 0.0, 0.0, 0.0)

    def __init__(self, meshcat, visible=Visible(), min_range=MinRange(),
                 max_range=MaxRange(), value=Value()):
        """
        Args:
            meshcat: A Meshcat instance.
            visible: An object with boolean elements for 'roll', 'pitch',
                     'yaw', 'x', 'y', 'z'; the intention is for this to be the
                     PoseSliders.Visible() namedtuple.  Defaults to all true.
            min_range, max_range, value: Objects with float values for 'roll',
                      'pitch', 'yaw', 'x', 'y', 'z'; the intention is for the
                      caller to use the PoseSliders.MinRange, MaxRange, and
                      Value namedtuples.  See those tuples for default values.
        """
        LeafSystem.__init__(self)
        port = self.DeclareAbstractOutputPort(
            "pose", lambda: AbstractValue.Make(RigidTransform()),
            self.DoCalcOutput)

        # The widgets themselves have undeclared state.  For now, we accept it,
        # and simply disable caching on the output port.
        # TODO(russt): consider implementing the more elaborate methods seen
        # in, e.g., LcmMessageSubscriber.
        port.disable_caching_by_default()

        self._meshcat = meshcat
        self._visible = visible
        self._value = list(value)
        
        for i in range(6):
            if visible[i]:
                meshcat.AddSlider(min=min_range[i],
                                 max=max_range[i],
                                 value=value[i],
                                 step=0.01,                                 name=value._fields[i])

    def __del__(self):
        for s in ['roll', 'pitch', 'yaw', 'x', 'y', 'z']:
            if visible[s]:
                self._meshcat.DeleteSlider(s)

    def SetPose(self, pose):
        """
        Sets the current value of the sliders.
        Args:
            pose: Any viable argument for the RigidTransform
                  constructor.
        """
        tf = RigidTransform(pose)
        self.SetRpy(RollPitchYaw(tf.rotation()))
        self.SetXyz(tf.translation())

    def SetRpy(self, rpy):
        """
        Sets the current value of the sliders for roll, pitch, and yaw.
        Args:
            rpy: An instance of drake.math.RollPitchYaw
        """
        self._value[0] = rpy.roll_angle()
        self._value[1] = rpy.pitch_angle()
        self._value[2] = rpy.yaw_angle()
        for i in range(3):
            if self._visible[i]:
                self._meshcat.SetSliderValue(
                    self._visible._fields[i], self._value[i])

    def SetXyz(self, xyz):
        """
        Sets the current value of the sliders for x, y, and z.
        Args:
            xyz: A 3 element iterable object with x, y, z.
        """
        self._value[3:] = xyz
        for i in range(3,6):
            if self._visible[i]:
                self._meshcat.SetSliderValue(
                    self._visible._fields[i], self._value[i])

    def DoCalcOutput(self, context, output):
        """
        Constructs the output values from the widget elements.
        """
        for i in range(6):
            if self._visible[i]:
                self._value[i] = self._meshcat.GetSliderValue(
                    self._visible._fields[i])
        output.set_value(RigidTransform(
            RollPitchYaw(self._value[0], self._value[1], self._value[2]),
            self._value[3:]))

class WsgButton(LeafSystem):
    def __init__(self, meshcat):
        LeafSystem.__init__(self)
        port = self.DeclareVectorOutputPort("wsg_position", 1,
            self.DoCalcOutput)
        port.disable_caching_by_default()
        self._meshcat = meshcat
        self._button = "Schunk WSG Open/Close"
        meshcat.AddButton(self._button)

    def __del__(self):
        self._meshcat.DeleteButton(self._button)

    def DoCalcOutput(self, context, output):
        position = 0.107  # open
        if (self._meshcat.GetButtonClicks(self._button) % 2) == 1:
            position = 0.002  # close
        output.SetAtIndex(0, position)
              


In [ ]:
def teleop_3d():
    builder = DiagramBuilder()

    station = builder.AddSystem(ManipulationStation())

    station.SetupClutterClearingStation()
    #ycb_objects = CreateClutterClearingYcbObjectList()
    #for model_file, X_WObject in ycb_objects:
    #    station.AddManipulandFromFile(model_file, X_WObject)
    station.AddManipulandFromFile(
        "drake/examples/manipulation_station/models/"
        + "061_foam_brick.sdf",
        RigidTransform(RotationMatrix.Identity(), [0, -0.6, 0.2]))
    station.Finalize()

    visualizer = MeshcatVisualizerCpp.AddToBuilder(
        builder, station.GetOutputPort("query_object"), meshcat)
    meshcat.ResetRenderMode()

    robot = station.get_controller_plant()
    params = DifferentialInverseKinematicsParameters(
        robot.num_positions(), robot.num_velocities())

    time_step = 0.005
    params.set_timestep(time_step)
    # True velocity limits for the IIWA14 (in rad, rounded down to the first
    # decimal)
    iiwa14_velocity_limits = np.array([1.4, 1.4, 1.7, 1.3, 2.2, 2.3, 2.3])
    params.set_joint_velocity_limits((-iiwa14_velocity_limits,
                                    iiwa14_velocity_limits))
    differential_ik = builder.AddSystem(DifferentialInverseKinematicsIntegrator(
            robot, robot.GetFrameByName("iiwa_link_7"), time_step, params))
    builder.Connect(differential_ik.get_output_port(),
                    station.GetInputPort("iiwa_position"))

    teleop = builder.AddSystem(MeshcatPoseSliders(meshcat,
        min_range = MeshcatPoseSliders.MinRange(
            roll=0, pitch=-0.5, yaw=-np.pi, x=-0.6, y=-0.8, z=0.0),
        max_range = MeshcatPoseSliders.MaxRange(
            roll=2*np.pi, pitch=np.pi, yaw=np.pi, x=0.8, y=0.3, z=1.1)
    ))
    builder.Connect(teleop.get_output_port(0), 
                    differential_ik.get_input_port())
    wsg_teleop = builder.AddSystem(WsgButton(meshcat))
    builder.Connect(wsg_teleop.get_output_port(0),
                    station.GetInputPort("wsg_position"))

    diagram = builder.Build()
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()

    station_context = station.GetMyMutableContextFromRoot(context)

    q0 = station.GetOutputPort("iiwa_position_measured").Eval(
        station_context)
    differential_ik.get_mutable_parameters().set_nominal_joint_position(q0)
    diff_ik_context = differential_ik.GetMyMutableContextFromRoot(context)
    differential_ik.SetPositions(diff_ik_context, q0)
    teleop.SetPose(differential_ik.ForwardKinematics(diff_ik_context))

    if running_as_notebook:  # Then we're not just running as a test on CI.
        simulator.set_target_realtime_rate(1.0)

        meshcat.AddButton("Stop Simulation")
        while meshcat.GetButtonClicks("Stop Simulation") < 1:
            simulator.AdvanceTo(simulator.get_context().get_time() + 2.0)
        meshcat.DeleteButton("Stop Simulation")
    
    else:
        simulator.AdvanceTo(0.1)

teleop_3d()

# Teleop Example (2D)

Many of the core concepts in this class can be studied in 2D instead of 3D.  And everything is simpler/cleaner there,  including teleoperation!

In [ ]:
def teleop_2d():
  builder = DiagramBuilder()

  station = builder.AddSystem(ManipulationStation())
  station.SetupPlanarIiwaStation()
  station.AddManipulandFromFile(
      "drake/examples/manipulation_station/models/"
      + "061_foam_brick.sdf",
      RigidTransform(RotationMatrix.Identity(), [0.6, 0, 0]))
  # TODO(russt): Add planar joint to brick
  station.Finalize()

  visualizer = MeshcatVisualizerCpp.AddToBuilder(
      builder, station.GetOutputPort("query_object"), meshcat)
  meshcat.Set2dRenderMode()

  robot = station.get_controller_plant()
  params = DifferentialInverseKinematicsParameters(
      robot.num_positions(), robot.num_velocities())

  time_step = 0.005
  params.set_timestep(time_step)
  iiwa14_velocity_limits = np.array([1.4, 1.3, 2.3])
  params.set_joint_velocity_limits((-iiwa14_velocity_limits,
                                    iiwa14_velocity_limits))
  # These constants are in body frame.
  params.set_end_effector_velocity_gain([1, 0, 0, 0, 1, 1])
  differential_ik = builder.AddSystem(
      DifferentialInverseKinematicsIntegrator(
          robot, robot.GetFrameByName("iiwa_link_7"), time_step, params))
  builder.Connect(differential_ik.get_output_port(),
                  station.GetInputPort("iiwa_position"))

  meshcat.DeleteAllButtonsAndSliders()
  teleop = builder.AddSystem(MeshcatPoseSliders(meshcat,
      min_range=MeshcatPoseSliders.MinRange(
          roll=0, x=-0.6, z=0.0),
      max_range=MeshcatPoseSliders.MaxRange(
          roll=2*np.pi, x=0.8, z=1.1),
      value=MeshcatPoseSliders.Value(pitch=0, yaw=0, y=0),
      visible=MeshcatPoseSliders.Visible(pitch=False, yaw=False, y=False) 
  ))
  builder.Connect(teleop.get_output_port(0), 
                  differential_ik.get_input_port())
  wsg_teleop = builder.AddSystem(WsgButton(meshcat))
  builder.Connect(wsg_teleop.get_output_port(0),
                  station.GetInputPort("wsg_position"))

  diagram = builder.Build()
  simulator = Simulator(diagram)
  context = simulator.get_mutable_context()
  station_context = station.GetMyMutableContextFromRoot(context)

  q0 = station.GetOutputPort("iiwa_position_measured").Eval(
      station_context)
  differential_ik.get_mutable_parameters().set_nominal_joint_position(q0)
  diff_ik_context = differential_ik.GetMyMutableContextFromRoot(context)
  differential_ik.SetPositions(diff_ik_context, q0)
  teleop.SetPose(differential_ik.ForwardKinematics(diff_ik_context))

  if running_as_notebook:  # Then we're not just running as a test on CI.
      simulator.set_target_realtime_rate(1.0)

      meshcat.AddButton("Stop Simulation")
      while meshcat.GetButtonClicks("Stop Simulation") < 1:
          simulator.AdvanceTo(simulator.get_context().get_time() + 2.0)
      meshcat.DeleteButton("Stop Simulation")

  else:
      simulator.AdvanceTo(0.1)

teleop_2d()
